In [1]:
import numpy as np
import os
import pickle
from plotting_utils import heatmap,annotate_heatmap
list_of_tasks = 'autoencoder curvature denoise edge2d edge3d \
keypoint2d keypoint3d colorization \
reshade rgb2depth rgb2mist rgb2sfnorm \
room_layout segment25d segment2d vanishing_point \
segmentsemantic class_1000 class_places inpainting_whole'

task_list = 'autoencoder curvature denoise edge2d edge3d \
keypoint2d keypoint3d  \
reshade rgb2depth rgb2mist rgb2sfnorm \
room_layout segment25d segment2d vanishing_point_well_defined \
segmentsemantic_rb class_1000'

task_list_refined = 'autoencoder curvature denoise edge2d edge3d \
keypoint2d keypoint3d  \
reshade rgb2depth rgb2mist rgb2sfnorm \
room_layout segment25d segment2d vanishing_point \
semantic_seg class_1000'

task_list = task_list.split()
task_list_refined = task_list_refined.split()

with open('./../all_affinities_tk.pkl', 'rb') as f:
    data = pickle.load(f)
    
t_affinity_matrix = np.zeros((len(task_list),len(task_list)))
for i,task1 in enumerate(task_list):
    for j,task2 in enumerate(task_list):
        t_affinity_matrix[i,j] = data[task1+"__"+task2]
        
with open('./../wins_vs_pixels_16k.pkl', 'rb') as f:
    data = pickle.load(f)
#print(data)
#print(data['win_rates']['segmentsemantic_rb'])
import numpy as np
t_winrate_matrix = np.zeros((len(task_list),len(task_list)))
for i,task1 in enumerate(task_list):
    for j,task2 in enumerate(task_list):
        t_winrate_matrix[i,j] = data[task1+"__"+task2]
print(t_winrate_matrix.shape)

(17, 17)


In [2]:
results_path = './../results/DDScomparison_taskonomy/nyuv2/'

In [3]:
from scipy.stats import spearmanr,pearsonr

kernel_affinities =  np.load(results_path+'kernels.npy',allow_pickle=True).item()
rdm_affinities = np.load(results_path+'rdms.npy',allow_pickle=True).item()

corr_value = {}
for kernel,kernel_affinity in kernel_affinities.items():
    corr_value[kernel] = {}
    for f_ablation,f_ablation_affinity in kernel_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_winrate_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] = temp
        corr_value[kernel][f_ablation] = corr_value_matrix
            

for rdm,rdm_affinity in rdm_affinities.items():
    corr_value[rdm] = {}
    for f_ablation,f_ablation_affinity in rdm_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_winrate_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] =  temp
        corr_value[rdm][f_ablation] = corr_value_matrix

In [4]:
max_corr = 0
mean_corr = {}
for dist,corr_dist in corr_value.items():
    mean_corr[dist] = {}
    for f_ablation,corr_f_ablation in corr_dist.items():
        mean_corr[dist][f_ablation] = round(np.mean(corr_f_ablation),3)
        if np.mean(corr_f_ablation) > max_corr:
            max_corr = np.mean(corr_f_ablation)
            max_combo = [dist,f_ablation]
print(max_corr,max_combo)

0.8422584604324396 ['linear', 'instance_norm']


In [5]:
import pandas as pd
dfObj = pd.DataFrame(mean_corr)
print(dfObj)

                 rbf    lap  linear  pearson  euclidean  cosine
None           0.784  0.782   0.654    0.806      0.721   0.828
centering      0.784  0.782   0.654    0.780      0.721   0.749
znorm          0.784  0.807   0.797    0.829      0.798   0.815
group_norm     0.821  0.800   0.825    0.825      0.820   0.825
instance_norm  0.830  0.835   0.842    0.842      0.839   0.842
layer_norm     0.813  0.807   0.806    0.806      0.807   0.806
batch_norm     0.792  0.808   0.737    0.833      0.791   0.808


In [6]:
mean_col = 0
mean_row = 0
for j,task in enumerate(task_list):
    temp_col,_ = spearmanr(t_winrate_matrix[:,j], t_affinity_matrix[:,j])
    temp_row,_ = spearmanr(t_winrate_matrix[j,:], t_affinity_matrix[j,:])
    mean_col+=temp_col/len(task_list)
    mean_row+=temp_row/len(task_list)
print(mean_col,mean_row)

0.9194340578036585 0.6765866545500134


In [7]:
from scipy.stats import spearmanr,pearsonr

kernel_affinities =  np.load(results_path+'kernels.npy',allow_pickle=True).item()
rdm_affinities = np.load(results_path+'rdms.npy',allow_pickle=True).item()

corr_value = {}
for kernel,kernel_affinity in kernel_affinities.items():
    corr_value[kernel] = {}
    for f_ablation,f_ablation_affinity in kernel_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_affinity_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] = temp
        corr_value[kernel][f_ablation] = corr_value_matrix
            

for rdm,rdm_affinity in rdm_affinities.items():
    corr_value[rdm] = {}
    for f_ablation,f_ablation_affinity in rdm_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_affinity_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] =  temp
        corr_value[rdm][f_ablation] = corr_value_matrix

In [8]:
max_corr = 0
mean_corr = {}
for dist,corr_dist in corr_value.items():
    mean_corr[dist] = {}
    for f_ablation,corr_f_ablation in corr_dist.items():
        mean_corr[dist][f_ablation] = round(np.mean(corr_f_ablation),3)
        if np.mean(corr_f_ablation) > max_corr:
            max_corr = np.mean(corr_f_ablation)
            max_combo = [dist,f_ablation]
print(max_corr,max_combo)

0.8640426758938871 ['linear', 'instance_norm']


In [9]:
import pandas as pd
dfObj = pd.DataFrame(mean_corr)
print(dfObj)

                 rbf    lap  linear  pearson  euclidean  cosine
None           0.799  0.803   0.678    0.840      0.741   0.860
centering      0.799  0.803   0.678    0.804      0.741   0.763
znorm          0.786  0.817   0.799    0.840      0.802   0.822
group_norm     0.855  0.839   0.853    0.853      0.853   0.853
instance_norm  0.853  0.857   0.864    0.864      0.863   0.864
layer_norm     0.846  0.840   0.840    0.840      0.842   0.840
batch_norm     0.796  0.816   0.747    0.849      0.798   0.808
